In [264]:
import pandas as pd
import numpy as np
from scipy.stats import skew
from scipy import signal
from tsfresh import extract_features
from tsfresh import extract_relevant_features
from sklearn.ensemble import RandomForestClassifier

from tsfresh.feature_extraction import ComprehensiveFCParameters, EfficientFCParameters, MinimalFCParameters

con = EfficientFCParameters()



airpods_df = pd.read_csv('airpodsdata.csv',  usecols=['ParticipantID', 'TrialID', 'Label', 'Device', 'Time','AccelerationX','AccelerationY'
        ,'AccelerationZ','RotationRateX','RotationRateY','RotationRateZ'])
airpods_df = airpods_df.rename(columns={'AccelerationX': 'airpodsaccelerationX', 'AccelerationY': 'airpodsaccelerationY', 'AccelerationZ': 'airpodsaccelerationZ','RotationRateX': 'airpodsgyroX' ,'RotationRateY': 'airpodsgyroY' ,'RotationRateZ': 'airpodsgyroZ' })
iphone_df = pd.read_csv('iphonedata.csv', usecols=['ParticipantID', 'TrialID', 'Label', 'Device', 'Time','AccelerationX','AccelerationY'
        ,'AccelerationZ','RotationRateX','RotationRateY','RotationRateZ','MagneticFieldX','MagneticFieldY'
        ,'MagneticFieldZ'])
iphone_df = iphone_df.rename(columns={'AccelerationX': 'iPhoneaccelerationX', 'AccelerationY': 'iPhoneaccelerationY', 'AccelerationZ': 'iPhoneaccelerationZ','RotationRateX': 'iPhonegyroX' ,'RotationRateY': 'iPhonegyroY' ,'RotationRateZ': 'iPhonegyroZ' })

airpods_df2 = pd.read_csv('airpodsdata2.csv',  usecols=['ParticipantID', 'TrialID', 'Label', 'Device', 'Time','AccelerationX','AccelerationY'
        ,'AccelerationZ','RotationRateX','RotationRateY','RotationRateZ'])
airpods_df2 = airpods_df2.rename(columns={'AccelerationX': 'airpodsaccelerationX', 'AccelerationY': 'airpodsaccelerationY', 'AccelerationZ': 'airpodsaccelerationZ','RotationRateX': 'airpodsgyroX' ,'RotationRateY': 'airpodsgyroY' ,'RotationRateZ': 'airpodsgyroZ' })
iphone_df2 = pd.read_csv('iphonedata2.csv', usecols=['ParticipantID', 'TrialID', 'Label', 'Device', 'Time','AccelerationX','AccelerationY'
        ,'AccelerationZ','RotationRateX','RotationRateY','RotationRateZ','MagneticFieldX','MagneticFieldY'
        ,'MagneticFieldZ'])
iphone_df2 = iphone_df2.rename(columns={'AccelerationX': 'iPhoneaccelerationX', 'AccelerationY': 'iPhoneaccelerationY', 'AccelerationZ': 'iPhoneaccelerationZ','RotationRateX': 'iPhonegyroX' ,'RotationRateY': 'iPhonegyroY' ,'RotationRateZ': 'iPhonegyroZ' })

#format data
airpods_df = pd.concat([ airpods_df,airpods_df2])
iphone_df = pd.concat([ iphone_df,iphone_df2])

# concatenate the two dataframes
df = pd.concat([ airpods_df,iphone_df])
df.to_csv('tg.csv', index=False)

# Replace the character immediately preceding the time with a space
df['Time'] = df['Time'].str.replace(' ', '')
# Add a space between the time and date
df['Time'] = df['Time'].str.replace(r'(\d{4}/\d{2}/\d{2})(\d{2}:\d{2}:\d{2}.\d{3})', r'\1 \2')
# convert the time column to datetime object
df['Time'] = pd.to_datetime(df['Time'])
# sort the dataframe by time column
df = df.sort_values(by='Time')


# Fill missing values with the previous non-missing row
df.fillna(method='bfill', inplace=True)
df.drop_duplicates(inplace=True)

# Drop duplicate rows
df.drop_duplicates(inplace=True)
# Save the cleaned dataframe to a new CSV file
df = df.iloc[::2, :]
df = df.fillna(0)

df = df.drop('Device', axis=1)

#df = df[df['Label'].isin(['hand right', 'hand left','hand up','hand down','hand clockwise circle','standing still'])]
#df = df[df['Label'].isin(['head right', 'head left','head up','head down','head clockwise circle','standing still'])]
#df = df[df['Label'].isin(['hand right, head right', 'hand left, head left','hand up, head up','hand down, head down','hand clockwise circle, head clockwise circle','standing still'])]


df = df.groupby(['TrialID','Label','ParticipantID'])


column_value=['airpodsaccelerationX,airpodsaccelerationY','airpodsaccelerationZ','airpodsgyroX','airpodsgyroY','airpodsgyroZ','iPhoneaccelerationX','iPhoneaccelerationY','iPhoneaccelerationZ','iPhonegyroX','iPhonegyroY','iPhonegyroX','MagneticFieldX','MagneticFieldY','MagneticFieldZ']



FileNotFoundError: [Errno 2] No such file or directory: 'airpodsdata.csv'

In [ ]:
import json
from dataclasses import asdict
from tsfresh import extract_features, select_features



# Define feature extraction parameters
extraction_settingds = EfficientFCParameters()

#extract features
features = []
for trial, label in df:
    label = label.drop('ParticipantID', axis=1)
    removelabel = label.drop('Label', axis=1)
    extractedFeatures = extract_features(removelabel, column_id='TrialID', column_sort='Time', default_fc_parameters=extraction_settingds)
    extractedFeatures['TrialID'] = trial[0]
    extractedFeatures['Label'] = trial[1]
    features.append(extractedFeatures)

extractedFeatures = pd.concat(features,axis=0)

display(extractedFeatures)


extractedFeatures = extractedFeatures.fillna(0)

#split into train and test 
xtrain = extractedFeatures[extractedFeatures['TrialID'] < 9]
xtrain = xtrain.drop('TrialID', axis=1)
ytrain = xtrain['Label']
xtrain = xtrain.drop('Label', axis=1)

xtest = extractedFeatures[extractedFeatures['TrialID'] >= 9]
xtest = xtest.drop('TrialID', axis=1)
ytest = xtest['Label']
xtest = xtest.drop('Label', axis=1)


relevant_features = select_features(xtrain, ytrain, n_jobs=4,n_significant=100)




Feature Extraction: 100%|███████████████████████| 15/15 [00:02<00:00,  6.39it/s]


Feature Extraction:   0%|                                | 0/15 [00:00<?, ?it/s]

In [ ]:
print(len(relevant_features.columns))
relevant_featurescols = relevant_features.columns


In [ ]:
# Save the relevant feature names to a text file
with open("/Users/shelleytruong/PycharmProjects/pythonProject2/relevant_featuresall.txt", "w") as f:
    f.write("\n".join(relevant_features))

relevant_featurescols = relevant_features.columns


In [ ]:
print("Accuracy:", score)
from sklearn.metrics import f1_score

# Instantiate the NearestCentroid model
clf = RandomForestClassifier()
# Train a final model using only the relevant features
xtrain = xtrain.loc[:, relevant_featurescols]

xtest = xtest.loc[:, relevant_featurescols]

# Train the model on the data
clf.fit(xtrain, ytrain)


# Evaluate the performance of the model on the test set
score = clf.score(xtest, ytest)

# Make predictions on the test data
ypred = clf.predict(xtest)

# Calculate the F1 score
f1 = f1_score(ytest, ypred, average='weighted')

print("F1 score: {:.2f}".format(f1))


In [263]:
import pickle
# Save the model to a file
with open('/Users/shelleytruong/PycharmProjects/pythonProject2/rfcall.pkl', 'wb') as f:
    pickle.dump(clf, f)